In [1]:
! pip install git+https://github.com/modestyachts/ImageNetV2_pytorch

from imagenetv2_pytorch import ImageNetV2Dataset
import clip
from dataset import VLImageDataset
import torch
from tqdm import tqdm
import sys
sys.path.insert(0,'..')
sys.path.insert(0,'.')
import torch
import torch.nn as nn
from tqdm import tqdm
import argparse
from importlib import reload,import_module
import multiprocessing
import os
import time
from itertools import product
sys.path.append("./PTQ4ViT")
import PTQ4ViT.utils.datasets as datasets
import PTQ4ViT.utils.net_wrap as net_wrap
from PTQ4ViT.utils.quant_calib import QuantCalibrator, HessianQuantCalibrator
from PTQ4ViT.utils.models import get_net
from dataset import VLImageDataset, VLLoader


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/modestyachts/ImageNetV2_pytorch to /tmp/pip-req-build-u_uvnf9y
  Running command git clone --filter=blob:none --quiet https://github.com/modestyachts/ImageNetV2_pytorch /tmp/pip-req-build-u_uvnf9y
  Resolved https://github.com/modestyachts/ImageNetV2_pytorch to commit 14d4456c39fe7f02a665544dd9fc37c1a5f8b635
  Preparing metadata (setup.py) ... done


/home/subuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['.', '..', '/mnt/c/Users/sunde/Desktop/DL-Project', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/subuntu/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', './PTQ4ViT', './PTQ4ViT/utils']


In [2]:
model, preprocess = clip.load("ViT-B/32", device="cuda")


In [3]:
def test_classification(net,test_loader,max_iteration=None, description=None):
    pos=0
    tot=0
    i = 0
    max_iteration = len(test_loader) if max_iteration is None else max_iteration
    with torch.no_grad():
        q=tqdm(test_loader, desc=description)
        for inp,target in q:
            i+=1
            inp=inp.cuda()
            target=target.cuda()
            out=net(inp)
            pos_num=torch.sum(out.argmax(1)==target).item()
            pos+=pos_num
            tot+=inp.size(0)
            q.set_postfix({"acc":pos/tot})
            if i >= max_iteration:
                break
    print(pos/tot)
    return pos/tot

In [4]:
def init_config(config_name):
    """initialize the config. Use reload to make sure it's fresh one!"""
    _,_,files =  next(os.walk("PTQ4Vit/configs"))
    if config_name+".py" in files:
        quant_cfg = import_module(f"configs.{config_name}")
    else:
        raise NotImplementedError(f"Invalid config name {config_name}")
    reload(quant_cfg)
    return quant_cfg

In [5]:
image_features, text_features, target = VLLoader(model, preprocess)

g = VLImageDataset(image_features, text_features, target)
calib_loader = torch.utils.data.DataLoader(g, batch_size=32, num_workers=1)




[9567 5382 8010 6719 4907 2300  196 6102 4578 1337 8407 1985 1784 7310
 5207 3157 2584 9639 6686 3232 4035 7140 1528 5253 8013 4905 3027 5901
 6461  131 4788 4952]


In [6]:
def experiment_basic(net='vit_base_patch16_384', config="PTQ4ViT"):
    """
    A basic testbench.
    """
    quant_cfg = init_config(config)
    net = model
    wrapped_modules = net_wrap.wrap_modules_in_net(net,quant_cfg)
    
    # g=datasets.ViTImageNetLoaderGenerator('imagenetv2-matched-frequency-format-val','imagenet',32,32,16,kwargs={"model":net})
    # test_loader=g.test_loader()
    # calib_loader=g.calib_loader(num=32)


    quant_calibrator = HessianQuantCalibrator(net,wrapped_modules,calib_loader,sequential=False,batch_size=4) # 16 is too big for ViT-L-16
    quant_calibrator.batching_quant_calib()
    
    # test_classification(net,test_loader)

experiment_basic()

out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
out_proj
c_fc
c_proj
Completed net wrap.
prepare parallel calibration for ['visual.conv1', 'visual.transformer.resblocks.0.attn.out_proj', 'visual.transformer.resblocks.0.mlp.c_fc', 'visual.transformer.resblocks.0.mlp.c_proj', 'visual.transformer.resblocks.1.attn.out_proj', 'visual.transformer.resblocks.1.mlp.c_fc', 'visual.transformer.resblocks.1.mlp.c_proj', 'visual.transformer.resblocks.2.attn.out_proj', 'visual.transformer.resblocks.2.mlp.c_fc', 'visual.transformer.resblocks.2.mlp.c_proj', 'visual.transforme

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/subuntu/.local/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/subuntu/.local/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/subuntu/.local/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/mnt/c/Users/sunde/Desktop/DL-Project/dataset.py", line 46, in __getitem__
    image = self.image_features[idx]
RuntimeError: CUDA error: initialization error
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to en

In [ ]:
experiment_basic()
